In [37]:
import json, requests, time
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('postgresql://postgres:argmax@pg:5432/postgres')

# Postgres vector search
We have seen that postgres is capable of running a k-nearest neighbor search with the cosine similarity metric

## Example
These are the 10 closest apps to the zero vector

In [38]:
vec = [0 for _ in range(1024)]
sql_query = f'''
SELECT
    "bundleId"
FROM app_vectors
ORDER BY embedding<=>'{json.dumps(vec)}'
LIMIT 10
'''
with engine.connect() as db_con:
    df = pd.read_sql(sql_query, con=db_con)

df

,bundleId
0,com.YayySAL.DodgeAgent
1,com.tintash.nailsalon
2,com.tilegarden.match3
3,com.AppIdeas.LevelUpRunner
4,1582745578
5,1569586264
6,com.kamilbilge.ropesavior3d
7,se.ace.fishinc
8,com.volt.dresstoimpress
9,com.loop.match3d


# Embedding new data
We can embed new data with this API call

In [32]:
query_text = "A farming game"
query_vec = requests.get("http://emb:8080/embed", params ={"text": query_text}).json()
print(query_vec)

[0.354161, 0.848121, -0.030284, -0.266633, -0.291369, -0.235802, -0.328547, 0.556077, 0.266265, 0.80911, 0.088049, 0.425911, 0.366686, 0.266709, -0.140682, -0.460998, -0.554848, -0.686548, -0.32312, 0.068696, 0.559805, 0.457103, -0.860526, -0.005353, -0.523179, 0.301315, -0.240179, -0.324041, 0.659639, 0.821582, -0.161332, 0.125691, -0.01421, -0.347641, -0.507953, -0.617259, 0.252902, -0.143336, -0.472049, -0.525402, 0.562244, -0.354457, 0.047799, -0.168605, 0.183767, -0.484756, 0.497892, -0.919021, 0.49797, -0.050206, 0.134188, -0.502712, -0.414337, -0.215174, -0.64883, -0.228308, -0.019863, 0.199011, -0.309924, 0.597036, 1.353649, -0.230922, 0.069312, -0.212216, 0.794984, 0.892149, -0.509315, -0.528928, -0.061134, -0.452766, -0.853116, 0.23235, -0.493771, -0.62472, 0.480888, 0.490589, 0.388387, -0.040939, -0.14579, 0.589986, 0.033822, 0.541735, 0.020558, -0.312139, -0.767122, 0.004014, 0.754478, 0.060997, -0.146129, -0.595552, -0.218311, 0.355091, -0.330841, 0.541868, 0.405484, 0.875

#  Embed the text data within Postgres?
You are were asked to extend Postgres, such that the following syntax would work
## Desired outcome

    SELECT
        "bundleId"
    FROM app_vectors
    ORDER BY embedding<=>embed('a farming game')
    LIMIT 10

## Steps to achieve this
1. Use the [pghttp](https://github.com/pramsey/pgsql-http) extension to query the embedding endpoint.
2. Create an [SQL Function](https://www.postgresql.org/docs/current/sql-createfunction.html).
3. Verify that the above `embed` function yields the same result as embedding in python
4. Verify that the `ORDER BY embedding<=>embed('a farming game')` clause works without erros.

## Submission
You have 3 hours to complete the assignment.

Feel free to use any tool you think is helpful (Google, ChatGPT, CoPilot, etc).


Once done please notify your interviewer.


In [52]:
from sqlalchemy import text

# First, drop the existing function
drop_function = """
DROP FUNCTION IF EXISTS embed(text);
"""

# Now, create the new function with the correct return type
create_new_embed_function = """
CREATE OR REPLACE FUNCTION embed(query_text TEXT)
RETURNS vector AS $$
DECLARE
    api_response http_response;
BEGIN
    SELECT * INTO api_response
    FROM http_get('http://emb:8080/embed?text=' || urlencode(query_text));
    
    IF api_response.status != 200 THEN
        RAISE EXCEPTION 'HTTP request failed with status %', api_response.status;
    END IF;
    
    RETURN (SELECT array_agg(value::real)
            FROM json_array_elements_text(api_response.content::json))::vector;
END;
$$ LANGUAGE plpgsql;
"""

# Execute the function drop and creation
with engine.connect() as connection:
    connection.execute(text(drop_function))
    connection.execute(text(create_new_embed_function))
    connection.commit()

print("Embed function recreated successfully")

# Verify the new type
check_new_type = """
SELECT pg_typeof(embed('a farming game')) as new_embed_type;
"""

Embed function recreated successfully


In [47]:
from sqlalchemy import text

# Check the type of the 'embedding' column in the app_vectors table
check_embedding_type = """
SELECT pg_typeof(embedding) as embedding_type
FROM app_vectors
LIMIT 1;
"""

# Check the type returned by our embed function
check_embed_type = """
SELECT pg_typeof(embed('a farming game')) as embed_type;
"""

with engine.connect() as connection:
    embedding_type = connection.execute(text(check_embedding_type)).fetchone()[0]
    embed_type = connection.execute(text(check_embed_type)).fetchone()[0]

print(f"Type of 'embedding' column: {embedding_type}")
print(f"Type returned by embed function: {embed_type}")

Type of 'embedding' column: vector
Type returned by embed function: real[]


In [62]:
from sqlalchemy import text

# Test the query
final_query = """
SELECT "bundleId"
FROM app_vectors
ORDER BY embedding<=>embed('a farming game')
LIMIT 10
"""

with engine.connect() as connection:
    result = connection.execute(text(final_query))
    
    # Print column names
    print("\nColumn names:")
    print(result.keys())
    
    print("\nQuery results:")
    for row in result:
        print(row[0]) 

print("Query executed successfully")


Column names:
RMKeyView(['bundleId'])

Query results:
se.ace.fishinc
se.ace.fishinc
se.ace.fishinc
se.ace.fishinc
se.ace.fishinc
1436213906
1436213906
1436213906
1436213906
1436213906
Query executed successfully
